In [1]:
import matplotlib.pyplot as plt
import model.temporary.testing_k_folds as driver
from sklearn.metrics import confusion_matrix

'''
- i = get classification probabilities
- determine the bins
- determine students that fit within their
respective bins
- determine those who are TP and FP
  (probably determined beforehand somehow)
- loop through student list, summing
  all the empirical risks and then getting
  the mean
- Plot

- The empirical risk curve will most likely
  not resemble the paper, so there might
  need to be changes.
'''

model_pipeline, X_test, y_test = driver.get_model_pipeline()
print('done')

Number of students on reduced lunch: 23
<class 'pandas.core.frame.DataFrame'>
Scoring used:  recall
Best parameters:  {}
Best/Mean score using best parameters:  0.4
[[12  2]
 [ 2  8]]
Percentage:  0.8333333333333334
24
done


In [2]:
class Bin:
    total_instances = 0
    real_positives = 0
    upper_range = 0
    lower_range = 0
    index = 0
    empirical_risk = 0

    def __init__(self, index, number_of_bins):
        ranges = 1 / number_of_bins
        self.upper_range = (index+1) * ranges
        self.lower_range = (self.upper_range - ranges) - 0.0001
        self.index = index

    def add_student(self, is_real_positive):
        self.total_instances += 1

        if is_real_positive:
            self.real_positives += 1

        self.empirical_risk = self.real_positives / self.total_instances

Each bin object in the bins array will hold information regarding the
distribution of students. More than that, it will hold information regarding
the number of true positives out of the true positive and negatives.

In [3]:
number_of_bins = 4
bins = [Bin(x, number_of_bins)for x in range(0,number_of_bins)]

for i in bins:
    print(i.lower_range)
    print(i.upper_range)

print(len(bins))

def add_to_bin(bin_array, probability, true_positive):
    # takes O(n) but shouldn't matter with such a small dataset
    # can use another search method, but this will do for now
    # if need be, I can use another method of doing so

    if probability > 1 or probability < 0:
        raise ValueError("Probability must be in range (0, 1)")
    
    for k in bin_array:
        #bins encompass values of lower range <= x < upper_range
        if probability < k.upper_range:
            k.add_student(true_positive)
            return

    raise RuntimeError("Could not find bin for a student. Bin array was \
                       initialized wrongly")

print("TESTING====")
print(bins[0].total_instances)

add_to_bin(bins, 0.11, True)
add_to_bin(bins, 0.276, True)
add_to_bin(bins, 0.25, True)
print(bins[0].total_instances)
print(bins[1].total_instances)

-0.0001
0.25
0.2499
0.5
0.4999
0.75
0.7499
1.0
4
TESTING====
0
1
2


In [4]:
import numpy as np

predictions = []
y_pred = [] #this is for the confusion matrix

print(type(X_test))
print(X_test.index)
print(X_test.iloc[1])
print(type(X_test.iloc[2]))

print(X_test.shape)
model_pipeline.predict(X_test.iloc[2].values.reshape(1,-1))

for i in range(X_test.shape[0]):
    student_data_point = X_test.iloc[i].values.reshape(1,-1)
    pred = model_pipeline.predict(student_data_point)
    y_pred_prob = model_pipeline.best_estimator_['model'].predict_proba(student_data_point)[:, 1]

    #whether a student was positive, regardless if false positive or true positive
    student_actually_positive = y_test.iloc[i] == True

    predictions.append({"prediction": pred[0],
                        "student_actually_positive": student_actually_positive,
                        "probability": y_pred_prob[0]})

    y_pred.append(pred)

matrix = confusion_matrix(y_test, np.asarray(y_pred).reshape(-1,1))
print(matrix)

<class 'pandas.core.frame.DataFrame'>
Int64Index([68, 77, 44, 27, 63, 73, 61, 55, 51, 34, 38, 33, 10, 59, 36,  2, 45,
            31, 43, 15, 65, 47, 19, 39],
           dtype='int64')
A8                                      13.0
Has_504_No                               1.0
Has_504_Yes                              0.0
Student on Free or Reduced Lunch_No      0.0
Student on Free or Reduced Lunch_Yes     1.0
IEP/Specialized_No                       1.0
IEP/Specialized_Yes                      0.0
Name: 77, dtype: float64
<class 'pandas.core.series.Series'>
(24, 7)
[[12  2]
 [ 2  8]]


In [5]:
#### INITIALIZING THE BIN ARRAY
bins = [Bin(x, number_of_bins)for x in range(0,number_of_bins)]

#COUNT NUMBER OF TRUE POSITIVES
count = 0
for i in predictions:
    if i['student_actually_positive']:
        count += 1

print(count)

for student_results in predictions:
    add_to_bin(bins, student_results['probability'], student_results['student_actually_positive'])

instances_count = 0
for i in bins:
    print("BIN %i" % i.index)
    instances_count += i.total_instances
    print("Instances: ", i.total_instances)
    print("Actual positives: ", i.real_positives)
    print("Empirical Risk of Bin: ", i.empirical_risk)

print(instances_count)

10
BIN 0
Instanes:  13
Actual positives:  2
Empirical Risk of Bin:  0.15384615384615385
BIN 1
Instanes:  1
Actual positives:  0
Empirical Risk of Bin:  0.0
BIN 2
Instanes:  8
Actual positives:  6
Empirical Risk of Bin:  0.75
BIN 3
Instanes:  2
Actual positives:  2
Empirical Risk of Bin:  1.0
24
